# Time Window Data Loader

以上我們學了一個時間序列資料基本的性值，並且也介紹最天真的幾種預測方式。

而訓練模型時，常會需要將資料作隨機抽取，以便讓模型更新時增加隨機性也可以稍微避免over fitting。

但是時間序列資料具局部關聯性，所以不能隨便抽樣離散的時間點，必須以time window為單位抽樣資料

這邊我們介紹如何使用tf.data.Dataset來組成這樣抽樣的Dataloader:
- tf.data.Dataset.window
- Window-wise tf.data.Dataset
- Complete Data Loader

**開始前先import必要套件**

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

from plotly import express as px

**畫圖的功能以及toy data產生器**

In [ ]:
def plot_series(time, series, start=0, end=None, labels=None, title=None):
    #  Visualizes time series data
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  series (list of array of int) - 時間點對應的資料列表，列表內時間序列數量為D，
    #                                  每筆資料長度為T，若非為列表則轉為列表
    #  start (int) - 開始的資料序(第幾筆)
    #  end (int) -   結束繪製的資料序(第幾筆)
    #  labels (list of strings)- 對於多時間序列或多維度的標註
    #  title (string)- 圖片標題

    # 若資料只有一筆，則轉為list
    if type(series) != list:
        series = [series]

    if not end:
        end = len(series[0])

    if labels:
        # 設立dictionary, 讓plotly畫訊號線時可以標註label
        dictionary = {"time": time}
        for idx, l in enumerate(labels):
            # 截斷資料，保留想看的部分，並分段紀錄於dictionary中
            dictionary.update({l: series[idx][start:end]})
        # 畫訊號線
        fig = px.line(dictionary,
                      x="time",
                      y=list(dictionary.keys())[1:],
                      width=1000,
                      height=400,
                      title=title)
    else:
        # 畫訊號線
        fig = px.line(x=time, y=series, width=1000, height=400, title=title)
    fig.show()


def trend(time, slope=0):
    # 產生合成水平直線資料，其長度與時間等長，直線趨勢與設定slope相同
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  slope (float) - 設定資料的傾斜程度與正負
    # Returns:
    #  series (array of float) -  產出slope 與設定相同的一條線

    series = slope * time

    return series


def seasonal_pattern(season_time, pattern_type='triangle'):
    # 產生某個特定pattern，
    # Args:
    #  season_time (array of float) - 周期內的時間點, 長度為T
    #  pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #  data_pattern (array of float) -  根據自訂函式產出特定的pattern

    # 用特定function生成pattern
    if pattern_type == 'triangle':
        data_pattern = np.where(season_time < 0.5,
                                season_time*2,
                                2-season_time*2)
    if pattern_type == 'cosine':
        data_pattern = np.cos(season_time*np.pi*2)

    return data_pattern


def seasonality(time, period, amplitude=1, phase=30, pattern_type='triangle'):
    # Repeats the same pattern at each period
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   period (int) - 週期長度，必小於T
    #   amplitude (float) - 序列幅度大小
    #   phase (int) - 相位，為遞移量，正的向左(提前)、負的向右(延後)
    #   pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #   data_pattern (array of float) - 有指定周期、振幅、相位、pattern後的time series

    # 將時間依週期重置為0
    season_time = ((time + phase) % period) / period

    # 產生週期性訊號並乘上幅度
    data_pattern = amplitude * seasonal_pattern(season_time, pattern_type)

    return data_pattern


def noise(time, noise_level=1, seed=None):
    # 合成雜訊，這邊用高斯雜訊，機率密度為常態分布
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   noise_level (float) - 雜訊大小
    #   seed (int) - 同樣的seed可以重現同樣的雜訊
    # Returns:
    #   noise (array of float) - 雜訊時間序列

    # 做一個基於某個seed的雜訊生成器
    rnd = np.random.RandomState(seed)

    # 生與time同長度的雜訊，並且乘上雜訊大小 (不乘的話，標準差是1)
    noise = rnd.randn(len(time)) * noise_level

    return noise


def toy_generation(time=np.arange(4 * 365),
                   bias=500.,
                   slope=0.1,
                   period=180,
                   amplitude=40.,
                   phase=30,
                   pattern_type='triangle',
                   noise_level=5.,
                   seed=2022):
    signal_series = bias\
                  + trend(time, slope)\
                  + seasonality(time,
                                period,
                                amplitude,
                                phase,
                                pattern_type)
    noise_series = noise(time, noise_level, seed)

    series = signal_series+noise_series
    return series

In [ ]:
def split(x, train_size):
    # 最簡單直接取前後，並且時間點也記得要切，我們直接立個function
    return x[..., :train_size], x[..., train_size:]


# 先合成資料
time = np.arange(4 * 365)  # 定義時間點
series_sample = toy_generation(time)  # 這就是我們合成出來的資料

time_train, time_test = split(time, 365*3)
series_train, series_test = split(series_sample, 365*3)

## tf.data.Dataset.window

這邊我們會使用tf.data API，

裡面用tf.data.Dataset這個套件，將資料作成一個生成器：每次丟出特定處理過的部分資料，並且轉為tf.Tensor型態。

詳細使用方法請參考我們Deep Learning章節。

In [ ]:
import tensorflow.data as tfd

In [ ]:
dataset = tfd.Dataset.range(6)
print('Original Dataset')
for d in dataset:
    print(d, d.numpy())  # 可以用.numpy()轉成numpy格式，方便印出來看

使用```.window(size=W,shift=B)```功能可以將原本data以```W```為單位輸出，並每次往右位移```B```個單位找第一個資料，所以每個window的起始點間距為B

$w[k,\tau]=y[B*k+\tau]$, for $\tau\in\{0,1,2,...,W\}$

<img src=https://i.imgur.com/krybbFp.png width=400 align=left>

切成好幾個dataset

In [ ]:
print('Windowed Datasets')
for ds in dataset.window(size=3, shift=2):
    print(ds)
    print([d.numpy() for d in ds])
    print('------')

用```drop_remainder=True```可以把不滿足window size的dataset丟掉

In [ ]:
print('Windowed Datasets')
for ds in dataset.window(size=3, shift=1, drop_remainder=True):
    print(ds)
    print([d.numpy() for d in ds])
    print('------')

## Window-wise tf.Dataset

但我們希望一次餵給模型的不是一個dataset還要迴圈，而是一串資料。

所以我們用```flat_map(mapfun)```處理剛剛的結果，它會將每個資料夾先經過指定處理```mapfun```後，輸出成一個tf.Tensor而不是資料集

In [ ]:
W = 4
dataset = tfd.Dataset.range(10)
win_ds = dataset.window(size=W, shift=1, drop_remainder=True)
win_ds = win_ds.flat_map(lambda ds: ds.batch(W))
for data in win_ds:
    print(data.numpy())

最後我們將指定windo-wize的input資料與forcast目標

**預測下一個內容**
* 輸入序列: window中的前W-1個資料
* forcast目標: window中最後1個資料

In [ ]:
ds = win_ds.map(lambda x: (x[:-1], x[-1:]))
for x, y in ds:
    print("x = ", x.numpy(), "y = ", y.numpy())

**預測下K個內容**
* 輸入序列: window中的前W-K個資料
* forcast目標: window中最後K個資料

In [ ]:
K = 2
ds = win_ds.map(lambda x: (x[:-K], x[-K:]))
for x, y in ds:
    print("x = ", x.numpy(), "y = ", y.numpy())

## Complete Data Loader

我們的目標是要產生一個可以作各種資料分配操控的data loader，我們現在已經可以一次生成一個x,y

而訓練目標通常以預測下一個資料為主，我們可以先把剛剛的dataset產生的包成function:

In [ ]:
def win_ar_ds(series, size, shift=1):
    # 輸出Window-wise Forcasting Dataset
    # Args:
    #   series (array of float) - 時序資料, 長度為T
    #   size (int) - Window大小
    #   shift (int) - 每個window起始點間距
    # Returns:
    #   (tf.data.Dataset(母類名稱，切確type為MapDataset)) - 一個一次生一個window的生成器
    ds = tfd.Dataset.from_tensor_slices(series)
    ds = ds.window(size=size+1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda ds: ds.batch(size+1))
    return ds.map(lambda x: (x[:-1], x[-1:]))

In [ ]:
train_ds = win_ar_ds(series_train, size=10)  # 切time series
time_ds = win_ar_ds(time_train, size=10)  # time那邊也可以切出來對照一下

In [ ]:
# 我們吐前三個資料出來看
xx = []
yy = []
time_xx = []
time_yy = []
for (time_x, time_y), (x, y) in zip(time_ds.take(3), train_ds.take(3)):
    time_xx.append(time_x.numpy())
    time_yy.append(time_y[0].numpy())
    xx.append(x.numpy())
    yy.append(y[0].numpy())

In [ ]:
for i in range(3):
    plot_series(time_xx[i], xx[i], title=f'label {time_yy[i]} ={yy[i]}')

可看出三串 windowed series具時間推移的關係

後面可以用tf.data.Dataset常用的一些 cache, prefetch等增加效率技巧

並且對dataset作shuffle以及batch

In [ ]:
train_ds = win_ar_ds(series_train, size=10)  # 切time series
train_loader = train_ds.cache().prefetch(-1).shuffle(1000).batch(16)

所以訓練的framework大概就長這樣:

In [ ]:
# run loader
for x, y in train_loader:
    # 1. 跑model
    # 2. 計算loss
    # 3. 更新模型
    break
print("x shape:", x.shape, x.dtype)
print("y shape:", y.shape, y.dtype)

或一些處理time series的tf.keras.Model 可以使用model.fit:

```model.fit(train_loader)```

後面我們就來train一些可以train的model

## Compare to original Regression Data Loader

時間序列在預測時也可以混入各種不同序列，例如時間本身，或者一些sine，cosine訊號，

只要認為具有提供資訊的價值都可以放進來，在迴歸時稱為regressor(迴歸因子)。

相較於window-wise prediction, 最簡單的時間序列regression就沒有window，而是靠著已研究過的regressor來擬和時間序列

In [ ]:
def regressor_ds(*regressors, series):
    # 輸出Window-wise Regressor Forcasting Dataset
    # Args:
    #   regressors (arguments of array of float) - 多個迴歸因子，每個長度為T
    #   series (array of float) - 預測對象，長度
    # Returns:
    #   (tf.data.Dataset(母類名稱，切確type為TensorSliceDataset))
    #   - 一次生regressors和time series的dataset

    ds = tfd.Dataset.from_tensor_slices((np.stack(regressors, -1), series))
    return ds

In [ ]:
cos_train = seasonality(time_train, 180, 40., 30, 'cosine')
triag_train = seasonality(time_train, 180, 40., 30, 'triangle')

train_ds_t = regressor_ds(time_train.astype("float64"),
                          cos_train, triag_train, series=series_train)

In [ ]:
for mix, y in train_ds_t.batch(360).shuffle(100):
    break
plot_series(mix[:, 0],
            [mix[:, 0], mix[:, 1]+500, mix[:, 2]+500, y],
            labels=['t', 'cosine', 'triangle', 'series'])